In [15]:
import torch
import random
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.nn import Module
from torchvision import datasets, transforms
from torch.autograd import Variable
import os
from pathlib import Path
import torchvision
import PIL
import matplotlib.pyplot as plt
from PIL import Image
from pathlib import Path
from sklearn.preprocessing import MinMaxScaler
import torchvision.transforms.functional as TF
from torchvision.utils import make_grid

In [16]:
# setting device on GPU if available, else CPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)

Using device: cuda


In [17]:
# Define data transformations
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5,), (0.5,))])

# Download and load the training set
trainset = torchvision.datasets.MNIST(root='./data', train=True,
                                        download=True, transform=transform)

trainloader = torch.utils.data.DataLoader(trainset, batch_size=32,
                                          shuffle=True, num_workers=2)

# Download and load the test set
testset = torchvision.datasets.MNIST(root='./data', train=False,
                                       download=True, transform=transform)

testloader = torch.utils.data.DataLoader(testset, batch_size=32,
                                         shuffle=False, num_workers=2)

In [18]:
class Net(Module):
    def __init__(self):
        super(Net, self).__init__()
        
        self.linear = torch.nn.Sequential(
            nn.Linear(784, 400),
            nn.ReLU(),
            nn.Linear(400, 200),
            nn.ReLU(),
            nn.Linear(200, 100),
            nn.ReLU(),
            nn.Linear(100, 50),
            nn.ReLU(),
            nn.Linear(50, 10)
        )

    def forward(self,x):
        z = torch.flatten(x, start_dim=1)
        z = self.linear(z)
        return z

In [19]:
model = Net()
model = model.to(device)

In [20]:
optimizer = torch.optim.SGD(model.parameters(), lr = 1e-2, momentum=0.95)
criterion = nn.CrossEntropyLoss()

In [ ]:
epochs = 100
losses = []
for epoch in range(epochs):
    cumulative_loss = 0
    for data in trainloader:
        image, label = data
        image = image.to(device)
        label = label.to(device)
        output = model(image)
        loss = criterion(output, label)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        cumulative_loss += loss.item()
    losses.append(cumulative_loss / len(trainloader))
    
plt.plot([i for i in range(0, epochs)], losses)
plt.xlabel("epochs")
plt.ylabel("Cross Entropy Loss")
plt.show()

In [ ]:
print(f"last training loss: {loss.item()}")

In [ ]:
model.eval()
correct = 0
total = 0

for image, label in testloader:
    # Forward pass
    image = image.to(device)
    label = label.to(device)
    with torch.no_grad():
        outputs = model(image)
        
    
    _, predicted = torch.max(outputs.data, 1)
    
    total += label.size(0)
    correct += (predicted == label).sum().item()

accuracy = 100 * correct / total
print(f"Test Accuracy: {accuracy}%")